## Реестр заблокированных политический ресурсов

Итак, первый шаг, это найти список запрещенки. Роскомнадзор его не предоставляет, зато есть сервис по мониторингу интернета OONI https://explorer.ooni.org/country/RU

Используем API оттуда чтобы получить список запрещенных сайтов.

In [14]:
# Данные
#
# https://api.ooni.io/api/_/website_networks?probe_cc=RU
# http://pravo.minjust.ru/extremist-materials?combine=&page=307

In [15]:
import requests

In [16]:
r = requests.get('https://api.ooni.io/api/_/website_networks?probe_cc=RU')

In [17]:
asn_list = [i['probe_asn'] for i in r.json()['results']]
len(asn_list)

307

Это идентификаторы провайдеров, по которым собирается статистика

In [18]:
asn_list

[12389,
 8402,
 25513,
 48642,
 42387,
 35807,
 42610,
 12714,
 3216,
 8790,
 8334,
 31163,
 21479,
 42668,
 25490,
 31213,
 51604,
 41668,
 44724,
 31257,
 41733,
 15774,
 15640,
 12668,
 8492,
 34533,
 20632,
 8359,
 35000,
 6856,
 42511,
 29194,
 50716,
 16345,
 42682,
 47165,
 31200,
 8615,
 47438,
 20485,
 8427,
 15582,
 25159,
 51547,
 42893,
 8580,
 56981,
 12737,
 50676,
 28890,
 58304,
 56724,
 49342,
 38917,
 34757,
 47655,
 196705,
 13335,
 204297,
 50240,
 35539,
 50923,
 51341,
 15493,
 48149,
 8331,
 24955,
 51659,
 44927,
 25086,
 8570,
 31133,
 42065,
 28891,
 2848,
 28840,
 8369,
 29124,
 39153,
 51813,
 41843,
 51570,
 50473,
 12958,
 43966,
 60192,
 206804,
 41330,
 203730,
 34300,
 43274,
 31224,
 60139,
 199524,
 6939,
 203337,
 12722,
 31364,
 44493,
 39578,
 49392,
 31214,
 50544,
 206873,
 42429,
 209476,
 15378,
 33991,
 49048,
 57128,
 41661,
 198367,
 202422,
 8732,
 28884,
 205569,
 197460,
 29226,
 35723,
 41691,
 29497,
 44587,
 50542,
 44030,
 16083,
 494

In [19]:
def get_asn(asn, limit, offset):
    link = "https://api.ooni.io/api/_/website_urls?probe_cc=RU&probe_asn={}&limit={}&offset={}".format(asn, limit, offset)
    r = requests.get(link)
    return [i for i in r.json()['results']]

In [20]:
from progress.bar import Bar

In [21]:
# links = {}

# for asn in Bar('Countdown', check_tty=False).iter(asn_list):
#     links[asn] = []
#     offset = 0
#     while True:
#         batch = get_asn(asn, 50, offset*50)
#         links[asn] += batch
#         if len(batch) < 50 or offset > 10:
#             break
#         offset += 1
    

In [22]:
import json
# with open('01-ooni-asn-links.json', 'w') as fp:
#     json.dump(links, fp,  indent=4)

f = open('01-ooni-asn-links.json')
links = json.load(f)

In [23]:
list(links.values())[0][0]

{'anomaly_count': 242,
 'confirmed_count': 242,
 'failure_count': 1,
 'input': 'http://khodorkovsky.ru/',
 'total_count': 243}

In [24]:
import itertools
from urllib.parse import urlparse

links_set = set([urlparse(link['input']).netloc for link in itertools.chain(*links.values()) if link['anomaly_count'] > 0])

## Список заблокированных ресурсов по версии OONI

In [25]:
links_set

{'milli-mejlis.blogspot.com',
 'www.mailinator.com',
 'kavkaz.tv',
 'www.doovi.com',
 'qip.ru',
 'www.passion.ru',
 'www.pokerroom.com',
 'www.madre.org',
 'www.nazarene.org',
 'new-rutor.org',
 '8.8.4.4',
 'web.unep.org',
 'cdt.org',
 'yandex.ru',
 'www.washingtontimes.com',
 'mastodon.sdf.org',
 'www.scepsis.ru',
 'www.wluml.org',
 'www.agentprovocateur.com',
 'www.bing-amp.com',
 'www.churchofeuthanasia.org',
 'www.lingeriebowl.com',
 'threema.ch',
 'www.severreal.org',
 'protectionline.org',
 'ria.ru',
 'medportal.ru',
 'polithexogen.ru',
 'ikhwanonline.com',
 'www.mnr.gov.cn',
 'www.patreon.com',
 'megatfile.cc',
 'chinadaily.com.cn',
 'www.exmormon.org',
 'bridges.torproject.org',
 'thesnipergodproxy.github.io',
 'hidemy.name',
 'www.rosbalt.ru',
 'archive.org',
 'beboo.ru',
 'www.harkatulmujahideen.org',
 'plus.im',
 'www.instagram.com',
 'mospat.ru',
 'www.webbox.com',
 'transequality.org',
 'www.fring.com',
 'freenetproject.org',
 'hindunet.org',
 'trashy.com',
 'www.bing.com'

Теперь попробуем для каждого построить временной ряд блокировок

In [26]:
# https://api.ooni.io/api/v1/measurements?limit=50&failure=false&probe_cc=RU&domain=www.deti-404.com&probe_asn=12389&test_name=web_connectivity&since=2020-01-12&until=2022-02-12&anomaly=true&offset=50


def get_blocked_url_data(asn, domain):
    all_items = []
    i = 0
    while True:
        url = "https://api.ooni.io/api/v1/measurements?limit=50&failure=false&probe_cc=RU&domain={}&probe_asn=12389&test_name=web_connectivity&since=2015-01-01&until=2022-02-12&anomaly=true&offset={}".format(domain, i*50)
        try:
            results = requests.get(url).json()['results']
        except:
            break

        all_items += results
        if len(results) < 50:
            break
        i += 1

    return all_items

Соберём данные по провайдеру 12389

In [27]:
data_by_domain = {}

for domain in Bar('BlocksByTime', check_tty=False).iter(links_set):
    data_by_domain[domain] = get_blocked_url_data(12389, domain)

BlocksByTime |################################| 1530/1530


In [29]:
with open('02-domain-by-time.json', 'w') as fp:
    json.dump(data_by_domain, fp,  indent=4)

In [32]:
import matplotlib as plt

for domain, data in data_by_domain.items():
    d = [d['measurement_start_time'] for d in data if d['anomaly'] == True]
    if len(data) > 0:
        print(domain, d[0], d[-1])
    

milli-mejlis.blogspot.com 2022-02-06T10:40:54Z 2021-10-18T03:27:19Z
www.mailinator.com 2022-01-20T07:17:04Z 2018-06-02T14:59:16Z
kavkaz.tv 2022-02-11T10:23:31Z 2017-07-09T14:38:49Z
www.doovi.com 2022-02-11T07:47:08Z 2019-04-12T20:21:18Z
qip.ru 2022-01-30T05:27:44Z 2021-09-24T07:34:14Z
www.passion.ru 2022-01-20T12:31:07Z 2021-02-23T15:53:11Z
www.pokerroom.com 2022-01-03T03:43:25Z 2017-03-26T17:55:54Z
www.madre.org 2022-01-10T09:30:50Z 2021-05-04T03:22:23Z
www.nazarene.org 2022-02-10T18:30:21Z 2021-05-30T14:46:21Z
new-rutor.org 2022-02-10T18:40:19Z 2018-08-18T22:35:08Z
8.8.4.4 2022-02-06T07:28:44Z 2021-12-04T22:07:18Z
web.unep.org 2022-01-19T03:28:04Z 2021-09-29T10:43:52Z
yandex.ru 2021-10-29T04:30:47Z 2021-09-26T22:57:26Z
www.washingtontimes.com 2022-01-10T08:08:22Z 2021-05-12T03:28:25Z
mastodon.sdf.org 2022-02-08T16:56:32Z 2021-10-01T22:01:58Z
www.scepsis.ru 2021-12-13T17:54:36Z 2021-12-10T03:35:19Z
www.wluml.org 2022-02-07T23:31:45Z 2021-05-16T06:48:07Z
www.agentprovocateur.com 2020-0